# IBM Data Sciecne Professional Final Project 

## CAPSTONE

In [97]:
# import problem
import numpy as np
import pandas as pd
import json
import geocoder
from geopy.geocoders import Nominatim 
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm 
import matplotlib.colors as colors
import sklearn 
from sklearn.cluster import KMeans
import folium
from fake_useragent import UserAgent

<h3> Step I Data Collection </h3>

In [98]:
nlist = []
data  = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Pune').text
soup =  BeautifulSoup(data, 'html.parser')
data1  = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Pimpri-Chinchwad').text
soup1 =  BeautifulSoup(data1, 'html.parser')

In [99]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    nlist.append(row.text)

In [100]:
for row in soup1.find_all("div", class_="mw-category")[0].findAll("li"):
    nlist.append(row.text)

In [101]:
df = pd.DataFrame({"Neighborhood": nlist})
print("DataFrame Shape", df.shape)
df = df.drop_duplicates(subset=None, keep='first', inplace=False)
print("Update DataFrame Shape after removing duplicates row", df.shape)

DataFrame Shape (71, 1)
Update DataFrame Shape after removing duplicates row (71, 1)


In [102]:
df2 = df.drop([27,41])
# deleted wrong contain

In [103]:
print(df.shape)

print(df2.shape)

(71, 1)
(69, 1)


In [104]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Pune, Maharashatra, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
coords = []

In [105]:
for neighborhood in df2["Neighborhood"].tolist():
    coords.append(get_latlng(neighborhood))


In [106]:
dfcord = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [107]:
df2['Latitude'] = dfcord['Latitude']
df2['Longitude'] = dfcord['Longitude']

In [108]:
df3 = df2.drop([68,69,70])

In [109]:
df3.to_csv("df_data.csv", index=False)

In [110]:
address = 'Pune, Maharashtra, India '

geolocator = Nominatim(user_agent="User-Agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Pune, Maharashtra, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Pune, Maharashtra, India 18.521428, 73.8544541.


In [111]:
map_pune = folium.Map(location=[latitude, longitude], zoom_start=10)

# adding markers to map
for lat, lng, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_pune)  
    
map_pune

In [112]:
map_pune.save('map_pune.html')

In [113]:
CLIENT_ID = 'RG4H2GV5W3KMNXIQZCWVHSMF2GWLVAVJV2NV0JUEMVM21P4G' 
CLIENT_SECRET = '1QBUYAR5IEGETXA2VCVGD2ZUDVF0H4I2BWW0LUMTSKZYJI2O' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RG4H2GV5W3KMNXIQZCWVHSMF2GWLVAVJV2NV0JUEMVM21P4G
CLIENT_SECRET:1QBUYAR5IEGETXA2VCVGD2ZUDVF0H4I2BWW0LUMTSKZYJI2O


In [114]:

radius = 3000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [ ]:
# convert the venues list into a new DataFrame
venue_df = pd.DataFrame(venues)

# define the column names
venue_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venue_df.shape)

(4371, 7)


In [ ]:
venue_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Appa Balwant Chowk,100,100,100,100,100,100
"Aundh, Pune",97,97,97,97,97,97
Balewadi,100,100,100,100,100,100
Baner,62,62,62,62,62,62
Bavdhan,51,51,51,51,51,51
...,...,...,...,...,...,...
Vadgaon Budruk,63,63,63,63,63,63
Vimannagar,59,59,59,59,59,59
Vishrantwadi,29,29,29,29,29,29


In [ ]:
print('There are',(len(venue_df['VenueCategory'].unique()) , 'uniques categories.'))

There are (156, 'uniques categories.')


In [ ]:
venue_df['VenueCategory'].unique()[0:10]

array(['Indian Restaurant', 'Ice Cream Shop', 'History Museum',
       'Donut Shop', 'Tea Room', 'Historic Site', 'Café', 'Juice Bar',
       'Vegetarian / Vegan Restaurant', 'BBQ Joint',
       'South Indian Restaurant', 'Bistro', 'Snack Place',
       'Fast Food Restaurant', 'Sandwich Place',
       'Maharashtrian Restaurant', 'Coffee Shop', 'Stadium',
       'Dessert Shop', 'Food Truck', 'Trail', 'Italian Restaurant',
       'Seafood Restaurant', 'Bakery', 'Gym', 'Bar', 'Supermarket',
       'Hotel', 'Steakhouse', 'Plaza', 'Burger Joint', 'Deli / Bodega',
       'Restaurant', 'Gym / Fitness Center', 'Sports Bar', 'Lounge',
       'Hotel Bar', 'Sporting Goods Shop', 'General Entertainment',
       'Chinese Restaurant', 'Theme Park', 'Garden', 'Frozen Yogurt Shop',
       'Hookah Bar', 'Smoke Shop', 'Gastropub', 'Pizza Place',
       'Shopping Mall', 'Bookstore', 'English Restaurant'], dtype=object)

In [ ]:
"Shopping Mall" in venue_df['VenueCategory'].unique()

True

One Hot encoding

In [ ]:
OH_kl = pd.get_dummies(venue_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
OH_kl['Neighborhoods'] = venue_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [OH_kl.columns[-1]] + list(OH_kl.columns[:-1])
OH_kl = OH_kl[fixed_columns]


In [ ]:
group_kl = OH_kl.groupby(["Neighborhoods"])
new_group_kl = group_kl.mean()
reset_new_group_kl = new_group_kl.reset_index()

print(reset_new_group_kl.shape)


(65, 157)


In [ ]:
ax =len(reset_new_group_kl[reset_new_group_kl["Shopping Mall"] > 0])
near_mall = reset_new_group_kl[["Neighborhoods","Shopping Mall"]]


In [ ]:
near_mall.head(10)

,Neighborhoods,Shopping Mall
0,Appa Balwant Chowk,0.010000
1,"Aundh, Pune",0.010309
2,Balewadi,0.010000
3,Baner,0.000000
4,Bavdhan,0.000000
5,"Bhavani Peth, Pune",0.034483
6,Bhosari,0.000000
7,Blue Ridge Town Pune,0.018182
8,"Budhwar Peth, Pune",0.010000
9,"Chakan, Pune",0.000000


In [ ]:
near_mall.info()
near_mall.max()
near_mall.min()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Neighborhoods  65 non-null     object 
 1   Shopping Mall  65 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.1+ KB


Neighborhoods    Appa Balwant Chowk
Shopping Mall                   0.0
dtype: object

In [ ]:
numC = 3
mall_cluster = near_mall.drop(["Neighborhoods"], 1)
kcluster = KMeans(n_clusters = numC, random_state = 0).fit(mall_cluster)
kcluster.labels_


array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0])

In [ ]:
near_mall_copy =  near_mall.copy()
near_mall_copy['cluster'] =   kcluster.labels_
near_mall_copy.head()



,Neighborhoods,Shopping Mall,cluster
0,Appa Balwant Chowk,0.010000,1
1,"Aundh, Pune",0.010309,1
2,Balewadi,0.010000,1
3,Baner,0.000000,1
4,Bavdhan,0.000000,1


In [ ]:
near_mall_copy.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
near_mall_copy.head()
print(near_mall_copy.shape)
near_mall_copy = near_mall_copy.join(df3.set_index("Neighborhood"), on="Neighborhood")
near_mall_copy.head()
near_mall_copy.shape

(65, 3)


In [ ]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(numC)
ys = [i+x+(i*x)**2 for i in range(numC)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(near_mall_copy['Latitude'], near_mall_copy['Longitude'], near_mall_copy['Neighborhood'], near_mall_copy['cluster']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
map_clusters.save('map_punefinal.html')

# Result

The results from the k-means clustering show that we categorize the neighbourhoods into 3
Clusters are based on the frequency of occurrence for “Shopping Mall”:


Cluster 0 [Red]: Areas with moderate number of shopping malls

Cluster 1 [Purple]: Areas with low number to no existence of shopping malls

Cluster 2 [Green]: Areas with high concentration of shopping malls